# Working with ISIC Images
Images are at the heart of the ISIC Archive, and this notebook
will give you a sense of what the ```isicarchive``` package
can do in terms of retrieving images from the archive, and the
kinds of methods you can apply to them.
## Preparation
We'll start again by creating an ```IsicApi``` object, which
allows us to access the various pieces of information and images
stored in the archive. The call to ```cache_images()``` will
then download information (and meta information) about all
images that are available--if you are logged in as a regular
user, you may have access to additional images! **If you're
using the ```cache_images()``` method, please use a different
cache folder if you want to work with both publicly available
and private images in separate sessions.**

Next, we'll select all images from the ```IsicApi``` object that
have a specific diagnosis, and then display those images.

In [ ]:
# Settings (please adapt to your needs!)
username = 'weberj3@mskcc.org'

# DO NOT PUT A PASSWORD into the document, it will be requested,
# or you can store it in a privately readable ~/.netrc file!

cache_folder = 'C:\\Users\\weberj3\\Documents\\ISIC'
# cache_folder = '/Data/MSKCC/isicarchive/cache'

# setting debug flag, this will print out every web-API operation
debug = True

# Preparation
from isicarchive.api import IsicApi
from isicarchive import func, imfunc
api = IsicApi(username, cache_folder=cache_folder, debug=debug)

In [ ]:
# caching images and segmentations; this can take a long time!!
# so you may want to skip this cell for now...
api.cache_images()
api.cache_segmentations()

In [ ]:
# list studies
api.studies

In [ ]:
# load a study and cache the images associated with it
study = api.study('ISIC Annotation Study - All Features')

In [ ]:
# load image and data
image = api.image(study.images[0])
image.load_image_data()
image.load_superpixels()
image.map_superpixels()

In [ ]:
# create superpixel outlines as SVG paths
superpixel_colors = func.superpixel_colors()
outline_svg = imfunc.superpixel_outlines(
    image.superpixels['map'],
    image.superpixels['idx'].shape,
    out_format='osvg',
    path_attribs=superpixel_colors)
outlines = imfunc.superpixel_outlines(
    image.superpixels['map'],
    image.superpixels['idx'].shape,
    out_format='osvgp',
    path_attribs=superpixel_colors)

# display one path
outlines[0]

In [ ]:
# demostrate how to color a single superpixel
image_data = image.data
image_shape = image_data.shape
image_data.shape = (image_shape[0] * image_shape[1], -1)
map = image.superpixels['map']
superpixel_index = 472
pixel_count = map[superpixel_index, -1]
superpixel_pixels = map[superpixel_index, 0:pixel_count]
image_data[superpixel_pixels, 0] = 255
image_data[superpixel_pixels, 1] = 0
image_data[superpixel_pixels, 2] = 0
image_data.shape = image_shape

import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(image_data)
plt.show()

In [2]:
# retrieve segmentation (supports image name, image ID (via additional request), and segmentation ID)
from isicarchive.api import IsicApi
api = IsicApi(debug=True)
segmentation = api.segmentation('ISIC_0000000')

Requesting https://isic-archive.com/api/v1/dataset with params: {'limit': 0, 'detail': 'true'}
Retrieved list with 13 items.
Requesting https://isic-archive.com/api/v1/study with params: {'limit': 0, 'detail': 'true'}
Retrieved list with 2 items.
Requesting https://isic-archive.com/api/v1/image with params: {'name': 'ISIC_0000000'}
Retrieved list with 1 items.
Requesting https://isic-archive.com/api/v1/segmentation/5436e3abbae478396759f0cf
Error occurred: 400
Requesting https://isic-archive.com/api/v1/segmentation with params: {'imageId': '5436e3abbae478396759f0cf'}
Retrieved list with 1 items.
Requesting https://isic-archive.com/api/v1/segmentation/5463934bbae47821f88025ad
Retrieved dict with 7 keys.


In [3]:
# get superpixels in image mask
sp_weights = segmentation.superpixels_in_mask()
sp_half_in = segmentation.superpixels_in_mask(0.5)
sp_full_in = segmentation.superpixels_in_mask(1.0)

# print sum
print('Number of superpixels: ' + str(len(sp_weights)))
print('Average coverage: ' + str(sum(sp_weights) / len(sp_weights)))
print('Superpixels at least half in mask: ' + str(len(sp_half_in)))
print('Superpixels fully in mask: ' + str(len(sp_full_in)))

Requesting https://isic-archive.com/api/v1//segmentation/5463934bbae47821f88025ad/mask
Retrieved 4590 bytes of content.
Requesting https://isic-archive.com/api/v1/image/5436e3abbae478396759f0cf
Retrieved dict with 9 keys.
Requesting https://isic-archive.com/api/v1//image/5436e3abbae478396759f0cf/superpixels
Retrieved 73938 bytes of content.
Number of superpixels: 996
Average coverage: 0.4655419826175136
Superpixels at least half in mask: 462
Superpixels fully in mask: 426
